In [ ]:
# 합성곱(cc.Conv2d) + 활성화 합수(nn.ReLU) + 맥스풀링(nn.MaxPoold2d)를 하나의 합성곱 층으로 봄
# 3개의 층을 직접 구현

import torch
import torch.nn as nn

In [ ]:
# 임의의 텐서 생성
# 배치 크기 x 채널 x 높이 x 너비 크기의 텐서

inputs = torch.Tensor(1, 1, 28, 28)
print("텐서의 크기 : {}".format(inputs.shape))

텐서의 크기 : torch.Size([1, 1, 28, 28])


In [ ]:
# 첫 번째 합성곱 층 구현
# 1 채널을 입력받아 32채널을 뽑아냄 (커널 사이즈 = 3)

conv1 = nn.Conv2d(1, 32, 3, padding = 1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [ ]:
# 두 번째 합성곱 층 구현
# 32채널을 입력받아 64채널을 뽑아냄(커널 사이즈 = 3)

conv2 = nn.Conv2d(32, 64, kernel_size = 3, padding = 1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [ ]:
# 맥스풀링 구현
# 정수 하나를 인자로 넣으면 커널 사이즈와 스트라이드가 해당값으로 지정

pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [ ]:
# 첫 번째 합성곱층을 통과시키고 그 후의 텐서 크기 확인

out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [ ]:
# 맥스풀링을 통과시키고 그 후의 텐서 크기 확인

out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [ ]:
# 두 번째 합성곱층에 통과시키고 그 후의 텐서 크기 확인

out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [ ]:
# 맥스풀링을 통과시키고 그 후의 텐서 크기 확인

out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [ ]:
# out의 첫 번째 차원 출력

out.size(0)

1

In [ ]:
# out의 두 번째 차원 출력

out.size(1)

64

In [ ]:
# out의 세 번째 차원 출력

out.size(2)

7

In [ ]:
# out의 네 번째 차원 출력

out.size(3)

7

In [ ]:
# view를 사용하여 텐서를 펼치는 작업
# 첫 번째 차원인 배치 차원은 그대로 두고 나머지는 펼치기

out = out.view(out.size(0), -1)
print(out.shape)

# 배치 차원을 제외하고 모두 하나의 차원으로 통합

torch.Size([1, 3136])


In [ ]:
# 전결합층(Fully-Connected layer) 통과
# 출력층으로 10개의 뉴런을 배치하여 10개 차원의 텐서로 변환

fc = nn.Linear(3136, 10) # input_dim = 3136, output_dim = 10
out = fc(out)
print(out.shape)

torch.Size([1, 10])


In [ ]:
# CNN으로 MNIST 분류

import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [ ]:
# GPU 사용 가능하다면 device가 cuda, 아니라면 cpu

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# 랜덤 시드 고정
# device가 CPU일 경우 랜덤 시드 고정

torch.manual_seed(777)

In [ ]:
# GPU 사용 가능할 경우 랜덤 시드 고정
# CPU와 GPU의 랜덤 넘버 생성기가 분리
# 따라서 상단의 코드만으로는 GPU에서 발생하는 무작위성을 제어하지 못함

if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [ ]:
# 학습에 사용할 파라미터 설정

learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [ ]:
# 데이터 로더를 사용하가 위해 데이터셋 정의

mnist_train = dsets.MNIST(root = "MNIST_data/", # 다운로드 경로 지정
                          train = True, # True로 지정하면 훈련 데이터로 다운로드
                          transform = transforms.ToTensor(), # 텐서로 변환
                          download = True)

mnist_test = dsets.MNIST(root = 'MNIST_data/', # 다운로드 경로 지정
                         train = False, # False를 지정하면 테스트 데이터로 다운로드
                         transform = transforms.ToTensor(), # 텐서로 변환
                         download = True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 58.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.74MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.4MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 3.83MB/s]


In [ ]:
# 데이터 로더를 사용하여 배치 크기 지정

data_loader = torch.utils.data.DataLoader(dataset = mnist_train, # mnist_train 데이터셋을 데이터로더에 연결
                                          batch_size = batch_size,# 각 배치에 포함될 데이터 샘플 수 지정
                                          shuffle = True, # 각 epoch마다 데이터셋 순서 무작위로 셔플
                                          # 모델이 데이터의 특정 순서에 의존하는 것 방지, 일반화 성능 향상
                                          drop_last = True) # 전체 데이터셋의 크기가 배치사이즈로 나눠떨어지지 않을 때 남는 배치 버림

In [ ]:
# 클래스로 모델 설계

class CNN(torch.nn.Module):

  def __init__(self):
    super(CNN, self).__init__()
    # 첫 번째 층
    # ImgIn shape = (?, 28, 28, 1)
    #   Conv      ->(?, 28, 28, 32)
    #   Pool      ->(?, 14, 14, 32)
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
    )

    # 두 번째 층
    # ImgIn shape = (?, 14, 14, 32)
    #   Conv      ->(?, 14, 14, 64)
    #   Pool      ->(?, 7, 7, 64)
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
    )

    # 전결합층(Fully_Connected Layer) 7 x 7 x 64 inputs -> 10 outputs
    self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias = True)

    # 전결합층 한정으로 가중치 초기화
    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0), -1) # 전결합층을 위해서 Flatten, 배치 차원을 제외하고 모두 하나의 차원으로 통합
    out = self.fc(out)
    return out

In [ ]:
# CNN 모델 정의

model = CNN().to(device)

In [ ]:
# 비용 함수와 옵티마이저 정의

criterion = torch.nn.CrossEntropyLoss().to(device) # 비용 함수에 소프트맥스 함수 포함 X
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
# 총 배치의 수 출력

total_batch = len(data_loader)
print("총 배치의 수: {}".format(total_batch))

총 배치의 수: 600


In [ ]:
# 코드는 training_epochs로 지정된 횟수만큼 반복되는 바깥쪽 루프로 시작
# 에포크는 전체 훈련 데이터셋을 한 번 완전히 통과하는 것을 의미

# 에포크 루프 내부에는 data_loader를 반복하는 안쪽 루프
# data_loader는 작은 배치로 훈련 데이터를 제공하며, 이는 딥 러닝 모델을 훈련할 때의 표준적인 방식
# 안쪽 루프의 각 반복에서 입력 이미지 (X)와 해당 레이블 (Y)의 배치가 검색

# 다음으로, 입력 데이터와 레이블은 연산을 위해 지정된 device(CPU 또는 GPU)로 이동

# 순방향 패스를 수행하기 전에 optimizer.zero_grad()가 호출
# PyTorch에서는 기본적으로 기울기가 누적되므로 이것이 중요
# 현재 배치에만 기반하여 업데이트가 이루어지도록 이전 반복의 기울기를 지워야 함

# hypothesis = model(X) 줄은 순방향 패스를 수행
# 입력 배치 X는 model에 공급되고, 모델은 예측값 (hypothesis) 세트를 생성

# cost = criterion(hypothesis, Y) 줄은 손실 (비용이라고도 함)을 계산
# criterion은 모델의 예측이 실제 레이블과 얼마나 일치하는지를 측정하는 손실 함수
# 이 경우 분류 작업에 적합한 CrossEntropyLoss가 사용됩니다.

# cost.backward()는 역방향 패스 (역전파)를 수행
# 이는 모델의 각 매개변수에 대한 손실의 기울기를 계산
# 이러한 기울기는 손실을 줄이기 위해 각 매개변수가 얼마나 변경되어야 하는지를 나타냄

# 마지막으로 optimizer.step()은 계산된 기울기를 기반으로 모델의 매개변수를 업데이트
# optimizer(이 경우 Adam)는 손실을 최소화하기 위해
# 매개변수를 조정하는 가장 좋은 방법을 결정하기 위해 최적화 알고리즘을 사용

# 안쪽 루프 내부에서 avg_cost는 현재 배치의 비용을 total_batch 수로 나눈 값을 더하여 업데이트
# 이는 에포크 동안의 평균 비용을 추적하는 데 도움

# 안쪽 루프가 완료되면 (즉, 에포크의 모든 배치가 처리되면), 코드는 해당 에포크의 평균 비용을 출력
# 이를 통해 훈련 진행 상황을 모니터링하고 모델이 학습되고 있는지 (즉, 비용이 감소하고 있는지) 확인

In [ ]:
# 모델 훈련

for epoch in range(training_epochs):
  avg_cost = 0 # 에포크당 평균 비용을 저장하기 위한 변수 초기화

  for X, Y in data_loader: # 미니 배치 단위로 데이터 꺼내옴. X는 입력 데이터, Y는 레이블
  # 이미지 데이터는 이미 (28 x 28) 크기를 가지므로, 별도의 reshape 필요 없음
  # 레이블 Y는 원-핫 인코딩이 아닌 정수형 클래스 레이블
    X = X.to(device) # 입력 데이터를 연산이 수행될 장치로 이동
    y = Y.to(device) # 레이블을 연산이 수행될 장치로 이동

    optimizer.zero_grad() # 옵티마이저의 기울기 초기화
    hypothesis = model(X) # 모델을 통해 예측값(hypothesis) 계산, 순전파
    cost = criterion(hypothesis, Y) # 예측값과 실제값 Y의 손실 계산
    cost.backward() # 역전파 연산을 통해 기울기 계산
    optimizer.step() # 옵티마이저를 통해 파라미터 업데이트

    avg_cost += cost / total_batch # 현재 배치의 비용을 전체 배치 수로 나누어 누적

    # 에포크가 끝날 때마다 평균 비용 출력
    print("[Epoch: {:>4}] cost = {:>.9}".format(epoch + 1, avg_cost))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[Epoch:    7] cost = 0.0134643316
[Epoch:    7] cost = 0.0134988325
[Epoch:    7] cost = 0.0135118878
[Epoch:    7] cost = 0.0135857183
[Epoch:    7] cost = 0.0135899698
[Epoch:    7] cost = 0.013596287
[Epoch:    7] cost = 0.0135993874
[Epoch:    7] cost = 0.0136525389
[Epoch:    7] cost = 0.0136861689
[Epoch:    7] cost = 0.0136903897
[Epoch:    7] cost = 0.0136963045
[Epoch:    7] cost = 0.013705885
[Epoch:    7] cost = 0.0137157692
[Epoch:    7] cost = 0.0137267588
[Epoch:    7] cost = 0.0137484949
[Epoch:    7] cost = 0.0137556829
[Epoch:    7] cost = 0.0137666138
[Epoch:    7] cost = 0.0138686541
[Epoch:    7] cost = 0.0138855278
[Epoch:    7] cost = 0.0139200427
[Epoch:    7] cost = 0.0140000647
[Epoch:    7] cost = 0.0140413493
[Epoch:    7] cost = 0.0140433284
[Epoch:    7] cost = 0.014064109
[Epoch:    7] cost = 0.0141024273
[Epoch:    7] cost = 0.0141181815
[Epoch:    7] cost = 0.0141728623
[Epoch:    7] cost = 0.0141919376
[Epoch:    7] c

In [ ]:
# 모델 테스트
# 학습을 진행하지 않을 것이므로 torch.nn_grad() 사용

with torch.no_grad():
  # 테스트 데이터를 모델에 입력하기 위한 준비
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device) # 테스트 데이터셋의 크기를 맞추고, 연산을 위한 장치로 이동
  # CNN 모델에 필요한 형태(배치 크기, 채널, 높이, 너비)로 변경하고, 데이터 타입을 float으로 변환
  Y_test = mnist_test.test_labels.to(device) # 테스트 데이터셋의 레이블을 연산을 위한 장치로 이동

  # 모델 예측 수행
  prediction = model(X_test) # 테스트 데이터에 대해 모델이 예측한 결과값

  # 예측 결과와 실제 레이블 비교
  # 텐서의 차원 1을 따라 가장 큰 값의 인덱스 검색 -> 각 이미지에 대해 예측된 클래스를 나타내는 인덱스
  corrext_prediction = torch.argmax(prediction, 1) == Y_test # 예측된 클래스와 실제 레이블이 일치하는지 확인

  # 정확도 계산
  accuracy = corrext_prediction.float().mean() # 정확도를 계산하기 위해 일치하는 예측의 평균을 구함
  print("Accuracy: ", accuracy.item()) # 정확도를 출력

/usr/local/lib/python3.11/dist-packages/torchvision/datasets/mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy:  0.984000027179718
